In [848]:
import re
import numpy as np
from collections import defaultdict

In [849]:
def load_scans(filename='day19_scanners.txt'):
    with open(filename) as infile:
        scanners = infile.read().split('\n\n')

    scans = {}
    for scanner_data in scanners:
        scanner_data = scanner_data.split('\n')
        n_scanner = int(re.findall('\d+', scanner_data[0])[0])
        data = np.array([[int(c) for c in r.split(',')]
                         for r in scanner_data[1:]])
        scans[n_scanner] = data
    scans = [scans[i] for i in range(len(scans))]
    return scans

In [850]:
scans = load_scans()
ex_scans = load_scans('day19_example.txt')

In [851]:
scans = ex_scans

# Part 1

In [858]:
for data in scans:
    print(data.shape)
    print(data.min(axis=0))
    print(data.max(axis=0))
    print()

(25, 3)
[-892 -824 -901]
[630 900 763]

(25, 3)
[-500 -763 -893]
[807 935 858]

(26, 3)
[-889 -892 -804]
[697 759 800]

(25, 3)
[-938 -903 -870]
[647 780 750]

(26, 3)
[-743 -632 -822]
[927 719 876]



### Find all coordinate transformations

In [859]:
# Must be a nicer way to do this
units = []
for i in range(3):
    tmp = np.array([0, 0, 0])
    tmp[i] += 1
    units.append(tmp)
units

[array([1, 0, 0]), array([0, 1, 0]), array([0, 0, 1])]

In [860]:
rotations = []
for xdir in range(3):
    for xsign in [1, -1]:
        for ydir in set(range(3)) - {xdir}:
            if ydir != xdir:
                for ysign in [1, -1]:
                    zdir = (set(range(3)) - {xdir, ydir}).pop()
                    if str(xdir)+str(ydir) in '0120':
                        zsign = xsign*ysign
                    else:
                        zsign = -xsign*ysign 
                    x = units[xdir] * xsign
                    y = units[ydir] * ysign
                    z = units[zdir] * zsign
                    rot = np.array([x,y,z])
                    rotations.append(rot)
                    #print(rot)
len({tuple(tuple(c) for c in r) for r in orientations})

24

In [861]:
vecs = [
    [-1,-1,1],
    [-2,-2,2],
    [-3,-3,3],
    [-2,-3,1],
    [5,6,-4],
    [8,0,7]
]

#for rot in rotations:
#    for vec in vecs:
#        print(rot.dot(vec))
#    print()

### Try brute force approach

In [862]:
s1 = 0
s2 = 1

data1 = scans[s1]
data2 = scans[s2]

In [863]:
set1 = {*(tuple(c for c in r) for r in data1)}
found_match = False

#for rot in rotations:
#    rotated = rot.dot(data2.T).T
#    mean_diff = (data1.mean(axis=0) - rotated.mean(axis=0)).astype(int)
#    for i in range(-100, 100):
#        for j in range(-100, 100):
#            for k in range(-100, 100):
#                offset = mean_diff + np.array([i, j, k])
#                set2 = {*(tuple(c for c in r) for r in rotated + offset)}
#                if len(set1 & set2) > 0:
#                    found_match = True
#                    print('found match')
#                    break
#            if found_match:
#                break
#        if found_match:
#            break

In [864]:
i,j,k

(2, 0, -58)

### That's too slow

In [865]:
data1 = ex_scans[0]
data2 = ex_scans[1]

In [866]:
offset = np.array([68,-1246,-43])

In [867]:
set1 = {*(tuple(c for c in r) for r in data1)}
for rot in rotations:
    rotated = rot.dot((data2).T).T
    set2 = {*(tuple(c for c in r) for r in rotated + offset)}
    if len(set1 & set2) > 0:
        found_match = True
        print('found match')
        break

found match


### Look for an invariant - distances to other beacons

In [868]:
def get_dists(data):
    dists = []
    for row in data:
        dd = {*(tuple(c for c in r) for r in data - row)}
        dists.append(dd - {(0,0,0)})
    return dists

In [869]:
overlaps = {}
for s1, data1 in enumerate(scans):
    for s2, data2 in list(enumerate(scans))[s1+1:]:
        #print(f'checking scanners {s1} and {s2}')
        dists1 = get_dists(data1)
        for rot in rotations:
            rotated = rot.dot((data2).T).T
            dists2 = get_dists(rotated)
            
            matches = []
            for i, dd1 in enumerate(dists1):
                for j, dd2 in enumerate(dists2):
                    if (overlap := len(dd1 & dd2)) > 10:
                        matches.append((i, j, overlap))
            if len(matches) >= 12:
                print(f"Overlap between scanners {s1} and {s2}")
                overlaps[(s1, s2)] = (matches, rot)

Overlap between scanners 0 and 1
Overlap between scanners 1 and 3
Overlap between scanners 1 and 4
Overlap between scanners 2 and 4


In [870]:
# Not the correct way to get all beacons
sum([len(ss) for ss in scans]) - sum([len(v) for v in overlaps.values()])

119

In [871]:
def get_correction(s1, s2, overlaps, scans=scans):
    reverse = False
    if s1 > s2:
        s1, s2 = s2, s1
        reverse = True
        
    if (s1, s2) not in overlaps:
        print('Not an overlapping pair')
        return None
    
    matches, rot = overlaps[(s1, s2)]
    bid1, bid2, _ = matches[0]
    b1 = scans[s1][bid1]
    b2 = scans[s2][bid2]
    offset = b1 - rot.dot(b2)
    
    if reverse:
        rot = np.linalg.inv(rot)
        offset = b2 - rot.dot(b1)
    
    rot = rot.copy()
    offset = offset.copy()
    def correction(data):
        corrected = rot.dot((data).T).T + offset
        return corrected
    
    return correction

In [872]:
#correction14_0 = get_correction(14, 0, overlaps)
#set14_0 = {*(tuple(c for c in r) for r in correction14_0(scans[0]))}
#set14_0 & {*(tuple(c for c in r) for r in scans[14])}

In [873]:
#correction0_14 = get_correction(0, 14, overlaps)
#set0_14 = {*(tuple(c for c in r) for r in correction0_14(scans[14]))}
#set0_14 & {*(tuple(c for c in r) for r in scans[0])}

In [874]:
corrections[19][4](corrections[4][19](np.eye(3)))

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [875]:
corrections = defaultdict(dict)
for s1, s2 in overlaps.keys():
    corrections[s1][s2] = get_correction(s1, s2, overlaps)
    corrections[s2][s1] = get_correction(s2, s1, overlaps)

### Traverse the corrections graph recursively

In [878]:
def get_overlap(data1, data2):
    set1 = {*(tuple(c for c in r) for r in data1)}
    set2 = {*(tuple(c for c in r) for r in data2)}
    return set1 & set2

In [879]:
def correct_back(cur):
    print(f'\nEntering {cur}')
    corrected = {}
    to_descend = []
    for sid, correction in corrections[cur].items():
        if sid in visited:
            print(f'Already done {sid}')
            continue
        else:
            print(f'Correcting from {sid} to {cur} directly')
            corrected[sid] = correction(scans[sid])
            visited.add(sid)
            to_descend.append(sid)
    print(len(corrected))
            
    for sid in to_descend:
        print(f'Descending to find corrections to {sid}')
        to_correct = correct_back(sid)
        print(f'Back at {cur}')
        print(f'Got back {len(to_correct)} corrections')
        for target, data in to_correct.items():
            print(f'Correcting {target} inherited from {sid} to {cur}')
            corrected[target] = corrections[cur][sid](data)
    print(len(corrected))
    print()
            
    return corrected

In [880]:
visited = {0}
new_scans = correct_back(0)
new_scans[0] = scans[0]


Entering 0
Correcting from 1 to 0 directly
1
Descending to find corrections to 1

Entering 1
Already done 0
Correcting from 3 to 1 directly
Correcting from 4 to 1 directly
2
Descending to find corrections to 3

Entering 3
Already done 1
0
0

Back at 1
Got back 0 corrections
Descending to find corrections to 4

Entering 4
Already done 1
Correcting from 2 to 4 directly
1
Descending to find corrections to 2

Entering 2
Already done 4
0
0

Back at 4
Got back 0 corrections
1

Back at 1
Got back 1 corrections
Correcting 2 inherited from 4 to 1
3

Back at 0
Got back 3 corrections
Correcting 3 inherited from 1 to 0
Correcting 4 inherited from 1 to 0
Correcting 2 inherited from 1 to 0
4



In [881]:
full_map = set()
for data in new_scans.values():
    full_map |= {*(tuple(c for c in r) for r in data)}

In [890]:
len(full_map)

79

# Part 2

In [882]:
def get_offset(s1, s2, overlaps, scans=scans):
    reverse = False
    if s1 > s2:
        s1, s2 = s2, s1
        reverse = True
        
    if (s1, s2) not in overlaps:
        print('Not an overlapping pair')
        return None
    
    matches, rot = overlaps[(s1, s2)]
    bid1, bid2, _ = matches[0]
    b1 = scans[s1][bid1]
    b2 = scans[s2][bid2]
    offset = b1 - rot.dot(b2)
    
    if reverse:
        rot = np.linalg.inv(rot)
        offset = b2 - rot.dot(b1)
    
    def correction(pos):
        corrected = rot.dot(pos) + offset
        return corrected
    
    return correction

In [883]:
offsets = defaultdict(dict)
for s1, s2 in overlaps.keys():
    offsets[s1][s2] = get_offset(s1, s2, overlaps)
    offsets[s2][s1] = get_offset(s2, s1, overlaps)

In [884]:
offsets[14]

{}

In [885]:
def get_positions(cur):
    print(f'\nEntering {cur}')
    positions = {}
    to_descend = []
    for sid, correction in offsets[cur].items():
        if sid in visited:
            print(f'Already done {sid}')
            continue
        else:
            print(f'Correcting from {sid} to {cur} directly')
            positions[sid] = correction(np.array((0,0,0)))
            #print(positions[sid])
            visited.add(sid)
            to_descend.append(sid)
    print(len(positions))
            
    for sid in to_descend:
        print(f'Descending to find offsets to {sid}')
        to_correct = get_positions(sid)
        print(f'Back at {cur}')
        print(f'Got back {len(to_correct)} offsets')
        for target, pos in to_correct.items():
            print(f'Correcting {target} inherited from {sid} to {cur}')
            positions[target] = offsets[cur][sid](pos)
    print(len(positions))
    print()
            
    return positions

In [886]:
visited = {0}
positions = get_positions(0)
positions[0] = np.array((0, 0, 0))


Entering 0
Correcting from 1 to 0 directly
1
Descending to find offsets to 1

Entering 1
Already done 0
Correcting from 3 to 1 directly
Correcting from 4 to 1 directly
2
Descending to find offsets to 3

Entering 3
Already done 1
0
0

Back at 1
Got back 0 offsets
Descending to find offsets to 4

Entering 4
Already done 1
Correcting from 2 to 4 directly
1
Descending to find offsets to 2

Entering 2
Already done 4
0
0

Back at 4
Got back 0 offsets
1

Back at 1
Got back 1 offsets
Correcting 2 inherited from 4 to 1
3

Back at 0
Got back 3 offsets
Correcting 3 inherited from 1 to 0
Correcting 4 inherited from 1 to 0
Correcting 2 inherited from 1 to 0
4



In [887]:
max_dist = 0
pair = None
for i, pos1 in positions.items():
    for j, pos2 in positions.items():
        if i != j:
            dist = sum(abs(pos1 - pos2))
            if dist > max_dist:
                max_dist = dist
                pair = i, j

In [888]:
pair, max_dist

((3, 2), 3621.0)